In [1]:
import pandas as pd
import numpy as np

#### The transcript file contains:
##### 1) Customer id changed to integer format
##### 2) Offer if changed to integer format
##### 3) The promotion channel is segregated
###### 4) Offer status is one-hot encoded

In [2]:
transcript = pd.read_csv("full_transcript.csv", index_col=0)

In [3]:
transcript.head()

person  offer_id  offer received  offer viewed  offer completed  \
0       0       4.0               1             1                1   
1       0       7.0               2             2                1   
2       0       8.0               1             1                0   
3       0       9.0               1             1                0   
4       1       0.0               1             0                0   

   transaction  reward  amount  difficulty  email  mobile  social  web  bogo  \
0            3     3.0    7.54           7      1       1       1    1     0   
1            2     2.0    2.63          10      2       2       2    2     0   
2            0     0.0    0.00          10      1       1       0    1     0   
3            1     0.0    0.35          20      1       0       0    1     0   
4            0     0.0    0.00           0      1       1       0    1     0   

   discount  informational  
0         1              0  
1         1              0  
2         1              0  
3         1              0  
4         0              1

In [4]:
transcript['offer_id'] = transcript['offer_id'].astype(int)


### Dataframe user_df contains use information- required for user-user similarity based recommendation systems

In [5]:
user_df = pd.read_csv("user_df_median.csv", index_col=1)
user_df = user_df.drop(columns=['Unnamed: 0'])


In [6]:
user_df_scaled = pd.read_csv("user_scaled.csv")


In [9]:
user_df.columns


Index(['offer received', 'offer viewed', 'offer completed', 'transaction',
       'reward_infl', 'amount_infl', 'email', 'mobile', 'social', 'web',
       'bogo', 'discount', 'informational', 'avg_difficulty', 'time_to_open',
       'time_to_complete', 'total_amount', 'amount_pct', 'age', 'gender',
       'income', 'seen_ratio', 'completed_ratio'],
      dtype='object')

In [10]:
user_df.head()

offer received  offer viewed  offer completed  transaction  \
person                                                               
0.0                5.0           5.0              2.0          6.0   
1.0                2.0           0.0              1.0          0.0   
2.0                2.0           2.0              0.0          0.0   
3.0                4.0           4.0              3.0          3.0   
4.0                5.0           3.0              0.0          1.0   

        reward_infl  amount_infl  email  mobile  social  web  ...  \
person                                                        ...   
0.0             5.0        10.52    5.0     4.0     3.0  5.0  ...   
1.0             5.0         0.00    2.0     2.0     0.0  2.0  ...   
2.0             0.0         0.00    2.0     2.0     1.0  1.0  ...   
3.0            20.0        69.28    4.0     4.0     3.0  2.0  ...   
4.0             0.0         1.09    5.0     2.0     1.0  4.0  ...   

        avg_difficulty  time_to_open  time_to_complete  total_amount  \
person                                                                 
0.0               12.0      0.083929          0.285714         20.40   
1.0                2.0      1.000000          1.000000         77.01   
2.0                2.0      0.000000          1.000000         14.30   
3.0                5.0      0.234127          0.785714        159.27   
4.0                7.0      0.012500          1.000000          4.65   

        amount_pct   age  gender    income  seen_ratio  completed_ratio  
person                                                                   
0.0       0.515686  55.0     2.0   64000.0         1.0             0.40  
1.0       0.000000  55.0     0.0  112000.0         0.0             0.50  
2.0       0.000000  55.0     2.0   64000.0         1.0             0.00  
3.0       0.434985  75.0     0.0  100000.0         1.0             0.75  
4.0       0.234409  55.0     2.0   64000.0         0.6             0.00  

[5 rows x 23 columns]

### The data is grouped based on offer_id 

In [11]:
transcript_offerid_sum = transcript.groupby(by='offer_id', as_index=True).sum()


In [12]:
transcript_offerid_sum

person  offer received  offer viewed  offer completed  \
offer_id                                                            
0         53804397            7617          4144                0   
1         54036962            7618          6687                0   
2         53856961            7677          4171             4354   
3         53362664            7571          7264             4296   
4         53491172            7646          7337             5156   
5         54326882            7658          6716             3688   
6         53330192            7593          7298             3331   
7         53327699            7597          7327             5317   
8         53751618            7632          4118             4017   
9         54180985            7668          2663             3420   

          transaction   reward    amount  difficulty  email  mobile  social  \
offer_id                                                                      
0                2782      0.0  39591.26           0   7617    7617       0   
1                4892      0.0  54291.40           0   7618    7618    7618   
2                2517  21770.0  26874.43       31775   7677    7677       0   
3                4881  21480.0  55226.14       31310   7571    7571    7571   
4                5442  15468.0  54806.01       44275   7646    7646    7646   
5                6661  36880.0  58728.75       63740   7658    7658    7658   
6                6075  33310.0  58246.78       63300   7593    7593    7593   
7                6536  10634.0  63546.65       63320   7597    7597    7597   
8                2634   8034.0  31817.55       62850   7632    7632       0   
9                1856  17100.0  26772.21      127480   7668       0       0   

           web  bogo  discount  informational  
offer_id                                       
0         7617     0         0           6331  
1            0     0         0           6320  
2         7677  6355         0              0  
3         7571  6262         0              0  
4         7646     0      6325              0  
5            0  6374         0              0  
6         7593  6330         0              0  
7         7597     0      6332              0  
8         7632     0      6285              0  
9         7668     0      6374              0

In [13]:
transcript_offerid_sum = transcript_offerid_sum[['offer received','offer viewed', 'amount']]


In [14]:
transcript_offerid_sum['view_ratio'] = transcript_offerid_sum['offer viewed'] / transcript_offerid_sum['offer received']
transcript_offerid_sum['amount_per_view'] = transcript_offerid_sum['amount'] / transcript_offerid_sum['offer viewed']

transcript_offerid_sum

C:\Users\user\AppData\Local\Temp\ipykernel_1416\3568346705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_offerid_sum['view_ratio'] = transcript_offerid_sum['offer viewed'] / transcript_offerid_sum['offer received']
C:\Users\user\AppData\Local\Temp\ipykernel_1416\3568346705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transcript_offerid_sum['amount_per_view'] = transcript_offerid_sum['amount'] / transcript_offerid_sum['offer viewed']


offer received  offer viewed    amount  view_ratio  amount_per_view
offer_id                                                                     
0                   7617          4144  39591.26    0.544046         9.553875
1                   7618          6687  54291.40    0.877789         8.118947
2                   7677          4171  26874.43    0.543311         6.443162
3                   7571          7264  55226.14    0.959451         7.602718
4                   7646          7337  54806.01    0.959587         7.469812
5                   7658          6716  58728.75    0.876991         8.744602
6                   7593          7298  58246.78    0.961148         7.981198
7                   7597          7327  63546.65    0.964460         8.672943
8                   7632          4118  31817.55    0.539570         7.726457
9                   7668          2663  26772.21    0.347287        10.053402

### Most popular view

In [15]:
def most_viewed_offer (transcript_offerid_sum, top_k=5):
    #get an upper bound estiamte
    top_k_max = transcript_offerid_sum.shape[0]
    
    #replace upper bound
    if top_k > top_k_max:
        print("can't return more items than unique offer id")
        print("function will return  the maximum possible: {}".format(top_k_max))
        top_k = top_k_max
    
    #sort first by view ratio, then by money made per view
    sorted_transcript = transcript_offerid_sum.sort_values(by=['view_ratio', 'amount_per_view'], ascending=False)
    
    top_offers = sorted_transcript.index.tolist()[:top_k]
    
    return top_offers

In [16]:
most_viewed_offer(transcript_offerid_sum, top_k=3)


[7, 6, 4]

In [17]:
most_viewed_offer(transcript_offerid_sum, top_k=4)


[7, 6, 4, 3]

In [18]:
def most_profit_offer (transcript_offerid_sum, top_k=5):
    top_k_max = transcript_offerid_sum.shape[0]
    
    if top_k > top_k_max:
        print("Can't return more items than unique offer id")
        print("function will return  the maximum possible: {}".format(top_k_max))
        top_k = top_k_max
        
    sorted_transcript = transcript_offerid_sum.sort_values(by=['amount_per_view', 'view_ratio'], ascending=False)
    
    top_offers = sorted_transcript.index.tolist()[:top_k]
    
    
    return top_offers

In [19]:

most_profit_offer (transcript_offerid_sum, top_k=3)

[9, 0, 5]

### Collaborative filtering

In [20]:
user_views = transcript[['person', 'offer_id', 'offer viewed']]
#shuffle the entries
user_views = user_views.sample(frac=1)

user_views.head()

person  offer_id  offer viewed
43302   11599         2             0
53292   14298         7             1
30092    8068         9             1
9480     2556         5             1
23040    6191         0             1

In [21]:
# Create user-by-item matrix
user_by_item = user_views.groupby(['person', 'offer_id'])['offer viewed'].max().unstack()


#drop rows that are entirely nan
user_by_item = user_by_item.dropna(how='all')

user_by_item.head()

offer_id    0    1    2    3    4    5   6    7    8    9
person                                                   
0         NaN  NaN  NaN  NaN  1.0  NaN NaN  2.0  1.0  1.0
1         0.0  NaN  0.0  NaN  NaN  NaN NaN  NaN  NaN  NaN
2         NaN  1.0  1.0  NaN  NaN  NaN NaN  NaN  NaN  NaN
3         NaN  1.0  1.0  1.0  NaN  1.0 NaN  NaN  NaN  NaN
4         1.0  0.0  NaN  NaN  NaN  NaN NaN  NaN  NaN  2.0

In [22]:
def get_offers_seen(user_by_item, user):
    #Get the offers seen by a user, sort by most viewed first
    
    
    #get seen offers
    user_offers_seen = user_by_item.loc[user][user_by_item.loc[user].isna()==False]
    
    #sort by descending order
    user_offers_seen = user_offers_seen.sort_values(ascending=False)
    
    #get values
    user_offers_seen = user_offers_seen.index.values.tolist()
    
    return user_offers_seen

In [23]:
def create_user_offer_dict(user_by_item):
    #dict key is user and values are offers seen
    offers_seen = dict()
    
    #for each user get the offers seen
    for user in user_by_item.index:
        offers_seen[user] = get_offers_seen(user_by_item, user)
        
    return offers_seen


In [24]:
offers_seen_dict = create_user_offer_dict(user_by_item)


In [25]:
offers_seen_dict[0]


[7, 4, 8, 9]

In [26]:
#Measure similarity

In [27]:
def compute_similarity_euclid(user1_id, user2_id,user_df=user_df):
    #get user rows
    user1 = user_df.loc[user1_id]
    user2 = user_df.loc[user2_id]
    
    #ge the difference
    diff = user1 - user2
    
    #drop nans
    diff= diff.dropna()
    
    #calculate the norm
    norm = np.linalg.norm(diff)
    
    return norm
    

In [28]:
compute_similarity_euclid(5, 32)


84.93585510237546

In [29]:
def compute_similarity_corr(user1_id, user2_id, user_df=user_df, method='pearson'):
    #make dataframe from user ids
    df_corr = user_df.loc[[user1_id, user2_id]]
    
    #get correlation and select term off the diagonal
    corr = df_corr.transpose().corr(method).iloc[0,1]
    
    return corr

In [30]:
compute_similarity_corr(42, 1)


0.9999982355499337

In [31]:
compute_similarity_corr(42, 1, method='kendall')


0.45560568900151466

In [32]:
def compute_similarity_cosine(user1_id, user2_id,user_df=user_df):
    #get user rows
    user1 = user_df.loc[user1_id]
    user2 = user_df.loc[user2_id]
    
    cos_dist = 1 - spatial.distance.cosine(user1, user2)
    
    
    return cos_dist
    
    

In [33]:
from scipy import sparse, spatial

compute_similarity_cosine(42, 77)


0.9999982548512846

In [34]:
def find_most_similar(user, user_df=user_df, method='pearson', corr='pearson'):
    #store similarities here
    sim_list= []

    
    if method =='pearson':
        for idx in user_df.index:
            sim = compute_similarity_corr(user, idx, user_df, corr)
            sim_list.append(sim)
        #convert to numpy array
        sim_list= np.array(sim_list)
        
        most_sim_users = sim_list.argsort()[::-1]
        #drop the first user
        most_sim_users = most_sim_users[1:]
        
    if method =='euclid':
        for idx in user_df.index:
            sim = compute_similarity_euclid(user, idx)
            sim_list.append(sim)
        #convert to numpy array
        sim_list= np.array(sim_list)
        
        most_sim_users = sim_list.argsort()
        #drop the first user
        most_sim_users = most_sim_users[1:]
        
        
        if method =='cosine':
            for idx in user_df.index:
                sim = compute_similarity_cosine(user, idx)
                sim_list.append(sim)
            #convert to numpy array
            sim_list= np.array(sim_list)

            most_sim_users = sim_list.argsort()[::-1]
            #drop the first user
            most_sim_users = most_sim_users[1:]

        
    return most_sim_users

In [62]:
#Make recommendation based on similarity
coded_offerid={'0':'3f207df678b143eea3cee63160fa8bed',
 1:'5a8bc65990b245e5a138643cd4eb9837',
 2:'9b98b8c7a33c4b65b9aebfe6a799e6d9',
 3:'f19421c1d4aa40978ebb69ca19b0e20d',
 4:'2298d6c36e964ae4a3e7e9706d1fb8c2',
 5:'ae264e3637204a6fb9bb56bc8210ddfd',
 6:'4d5c57ea9a6940dd891ad53e9dbe8da0',
 7:'fafdcd668e3743c1bb461111dcafc2a4',
 8:'2906b810c7d4411798c6938adc9daaa5',
 9:'0b1e1539f2cc45b7b9fa7c272da2e1d7'}

In [65]:
def make_collab_recomm(user, offers_seen_dict = offers_seen_dict,top_k = 2, user_by_item = user_by_item,user_df = user_df, method='pearson', verbose=True):
    #find the most similar users
    sim_to_user = find_most_similar(user, user_df, method)
    
    #get offers seen
    idx_seen = offers_seen_dict[user]
    
    if verbose:
        print("Similarity measured by {}".format(method))
        print("Original user {} already saw: \t {}".format(user, idx_seen))
    
    #get number of offers already seen
    idx_seen_len = len(idx_seen)
    
    #get number of offers available
    len_available = user_by_item.shape[1]
    
    if top_k > (len_available - idx_seen_len):
        print("Top k of {} is bigger than possible, will return only {} offers".format(top_k, (len_available - idx_seen_len)))
        top_k = (len_available - idx_seen_len)
        
    #empty set to store unique offer ids
    proposed_offers = np.array([])
        
    for idx in sim_to_user:
        idx_sim_offer = offers_seen_dict[idx]
        
        new_offers = (np.setdiff1d(idx_sim_offer,idx_seen))
        
        if verbose:
            print("Similar user {} saw: \t {}".format(idx, idx_sim_offer))
            print("Unseen offers by original user: \t {}".format(new_offers))
        
        proposed_offers = np.unique(np.concatenate([new_offers, proposed_offers], axis=0))
        print("We propose so far: {}".format(proposed_offers))
            
        
        if len(proposed_offers)>top_k:
            break
    
    proposed_offers = proposed_offers[0:top_k]
    p=[]
    for i in proposed_offers:
        p.append(coded_offerid[i])
    if verbose:
        print("\nFor user {} we suggest offers : \t {}".format(user, p))
    
    return proposed_offers
        

In [66]:
make_collab_recomm(5, top_k=3, verbose=True)


Similarity measured by pearson
Original user 5 already saw: 	 [2, 7, 8, 0]
Similar user 6534 saw: 	 [0, 3, 6, 8]
Unseen offers by original user: 	 [3 6]
We propose so far: [3. 6.]
Similar user 1307 saw: 	 [3, 6, 7, 2]
Unseen offers by original user: 	 [3 6]
We propose so far: [3. 6.]
Similar user 98 saw: 	 [5, 1, 9]
Unseen offers by original user: 	 [1 5 9]
We propose so far: [1. 3. 5. 6. 9.]

For user 5 we suggest offers : 	 ['5a8bc65990b245e5a138643cd4eb9837', 'f19421c1d4aa40978ebb69ca19b0e20d', 'ae264e3637204a6fb9bb56bc8210ddfd']


array([1., 3., 5.])

In [67]:
make_collab_recomm(5, top_k=3, method='euclid',verbose=True)


Similarity measured by euclid
Original user 5 already saw: 	 [2, 7, 8, 0]
Similar user 440 saw: 	 [3, 6, 2]
Unseen offers by original user: 	 [3 6]
We propose so far: [3. 6.]
Similar user 12411 saw: 	 [1, 6, 8]
Unseen offers by original user: 	 [1 6]
We propose so far: [1. 3. 6.]
Similar user 10854 saw: 	 [7, 2, 4, 6]
Unseen offers by original user: 	 [4 6]
We propose so far: [1. 3. 4. 6.]

For user 5 we suggest offers : 	 ['5a8bc65990b245e5a138643cd4eb9837', 'f19421c1d4aa40978ebb69ca19b0e20d', '2298d6c36e964ae4a3e7e9706d1fb8c2']


array([1., 3., 4.])